In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Competition Objective
One liner - We are required to build an algorithm that can find our what are the datasets that a publications uses.

Description - In this competition, we need to develop an algorithm to automate the discovery of how scientific data are referenced in publications. We have with us the full text of scientific publications from numerous research areas, we'll identify data sets that the publications' authors used in their work.

We have a labelled dataset (train set) that we'll use to develop our algorithm. The unlabelled dataset (test set) will be used for evaluation of the algorithm.

In [ ]:
import pandas as pd
import os
os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data')

Data provided to us in this competition comprises of following 4 items:

* train.csv - This file comprises of publication-id and the associated data title and labels along with the cleansed label.
* train folder - This folder provides json file for each of the id's present in the above csv file, with each of the json file describing section-details of the publication and the associated text with it.
* sample_submission.csv - This file comprises of id, for which we must predict the data-set title.
* test folder - This folder provides json file for each of the id's present in test.csv file and the json file describes the section details of the publication and the associated text.

In [ ]:
import json
with open('../input/coleridgeinitiative-show-us-the-data/train/f8b03c87-9d1a-4f20-b76b-cb6c69d447b2.json') as f:
    sample = json.load(f)

In [ ]:
sample[:2]

In [ ]:
for s in sample:
    print(s['section_title'])

In [ ]:
import pandas as pd
df_train=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
for col in df_train.columns:
    print(f"{col}: {len(df_train[col].unique())}")

In [ ]:
df_test=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
df_test

In [ ]:
df_input = pd.DataFrame(columns=['id','section_title','text','data_label'])
ids=df_train['Id'].values
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
len(ids)

In [ ]:
for id in ids:
    df=pd.read_json ('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(id))
    
    for data_label in df_train[df_train['Id']==id]['dataset_label'].values:
        new_df=df[df['text'].str.contains(data_label)].copy(deep=True)
        new_df.loc[:,['data_label']] = data_label
        new_df.loc[:,['id']] = id
        new_df.reset_index(inplace=True,drop=True)
        df_input=pd.concat([df_input, new_df], ignore_index=True)
        df_input.reset_index(inplace=True,drop=True)

In [ ]:
df

In [ ]:
df_input.head()

In [ ]:
df_input.isnull().sum()

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import pickle
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
words =list( df_input['data_label'].values)
words=[word.split() for word in words]

In [ ]:
allwords = []
for wordlist in words:
    allwords += wordlist
#print(allwords)

In [ ]:
mostcommon = FreqDist(allwords).most_common(100)
#Taking most 100 frequent words occuring

wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))

fig = plt.figure(figsize=(30,10), facecolor='grey')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words in Data Label', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Freq of 25 Most Common Words in Data-Label', fontsize=60)
plt.show()

In [ ]:
words =list( df_input['section_title'].values)

stopwords=['ourselves', 'hers','the', 'between', 'yourself', 'but', 'again','of', 'there', 'about', 'once', 
           'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 
           'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 
           'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 
           'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 
           'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 
           'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 
           'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 
           'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 
           'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 
           'was', 'here', 'than']

split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopwords:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist
    
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words in Section Title', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Freq of 25 Most Common Words in Section-Title', fontsize=60)
plt.show()

In [ ]:
df_test_input = pd.DataFrame(columns=['id','section_title','text'])

ids=df_test['Id'].values
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
#len(ids)

In [ ]:
for id in ids:
    df=pd.read_json ('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(id))
    
    df.loc[:,['id']] = id
    df.reset_index(inplace=True,drop=True)
    df_test_input=pd.concat([df_test_input, df], ignore_index=True)
    df_test_input.reset_index(inplace=True,drop=True)

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
submission_df

In [ ]:
df_test_input

In [ ]:
df_test_input.text.str.len()

In [ ]:
df_test_input['length'] = df_test_input.text.str.len()

In [ ]:
df_test_input

In [ ]:
df_test_input =df_test_input[df_test_input.length > 0]
df_test_input

In [ ]:
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
submission_df.head()

In [ ]:
datasets_titles = [str(x).lower() for x in df_input['data_label'].unique()]
datasets_titles[0]

In [ ]:
import re
labels = []

In [ ]:
for index in submission_df.index:
    publication_text = df_test_input[df_test_input['id'] == index].text.str.cat(sep='\n').lower()
    
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append('|'.join(label))

In [ ]:
labels

In [ ]:
submission_df['PredictionString'] = labels

In [ ]:
submission_df.to_csv('submission.csv')

submission_df